In [12]:
import yfinance as yf
import numpy as np

# Fetch stock data using yfinance library
stock = yf.Ticker("AAPL")
df = stock.history(period="max")

# Extract the relevant columns (e.g., close price, volume)
data = df[["Open", "High", "Low", "Volume", "Close", ]].values

# Save the data as a numpy array
np.save("stock_data_with_volume.npy", data)

In [17]:
import torch
import torch.nn as nn

class TradingStrategy(nn.Module):
    def __init__(self):
        super(TradingStrategy, self).__init__()
        self.l1 = nn.Linear(4, 16) # Input layer with 3 features and 16 hidden units
        self.l2 = nn.Linear(16, 1) # Output layer with 1 unit

    def forward(self, x):
        x = torch.sigmoid(self.l1(x)) # Apply sigmoid activation function
        x = self.l2(x)
        return x

# Load the stock data with volume as a numpy array
data = np.load("stock_data_with_volume.npy")

# Define the cost function
def cost_function(net, data, target):
    prediction = net(data)
    cost = torch.mean((prediction - target)**2)
    return cost

# Initialize the network and the optimizer
net = TradingStrategy()
optimizer = torch.optim.Adam(net.parameters())

# Training loop
for i in range(1000):
    # Randomly shuffle the data
    np.random.shuffle(data)

    # Split the data into inputs (X) and target (y)
    X = data[:, :4]
    y = data[:, 4]

    # Convert the data to PyTorch tensors
    X = torch.from_numpy(X).float()
    y = torch.from_numpy(y).float()

    # Zero the gradients
    optimizer.zero_grad()

    # Forward pass
    output = net(X)

    # Compute the cost
    cost = cost_function(net, X, y)

    # Backward pass and optimization
    cost.backward()
    optimizer.step()
